PLS_VIP (From. 하랑)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# PLS 모델로부터 VIP 점수를 계산하는 함수 정의
def calculate_vip(pls_model):
    T = pls_model.x_scores_      # X score 행렬 (샘플별 컴포넌트), 샘플 수 x 컴포넌트 수
    W = pls_model.x_weights_     # X weight 행렬 (X의 feature가 컴포넌트를 만드는데 기여한 정도) 특성 수 x 컴포넌트 수
    Q = pls_model.y_loadings_    # Y 로딩 행렬 (컴포넌트가 Y를 설명하는데 얼마나 기여하는가) Y 변수 수 x 컴포넌트 수

    p, h = W.shape                # p: X feature 수, h: 컴포넌트 수


    # 컴포넌트별 설명된 분산(SS)을 계산
    SS = np.sum((T ** 2) @ (Q.T ** 2), axis=0)

    vip_scores = np.zeros(p)     # VIP 점수 저장 배열 초기화
    for i in range(p):           # 각 특성에 대해 반복
        weight = np.array([(W[i, j] ** 2) / np.sum(W[:, j] ** 2) for j in range(h)])  # 특성별 중요도 비율 계산
        vip_scores[i] = np.sqrt(p * np.sum(SS * weight) / np.sum(SS))                 # VIP 점수 계산
    return vip_scores


# 전체 데이터로 PLS 학습 및 VIP 계산
X_all = []
y_all = []

# 각 그룹별 X와 y 데이터 추출
for group, features_df in feature_dict.items():
    X_group = features_df.values  # 각 그룹의 특성 데이터
    y_group = np.array(weights_final[group])[:, 1] # 각 그룹의 wnpeights

    # 전체 X, y 데이터 합치기
    X_all.append(X_group)
    y_all.append(y_group)

# X_all, y_all 합치기
X_all = np.concatenate(X_all, axis=0)
y_all = np.concatenate(y_all, axis=0)

# PLS 모델 학습 (전체 데이터를 사용)
pls = PLSRegression(n_components=2)
pls.fit(X_all, y_all)

# VIP 계산
vip_scores = calculate_vip(pls)

# VIP 점수를 기준으로 중요한 피처 선택 (0.9 이상)
vip_df = pd.DataFrame({
    'Feature': features_df.columns,
    'VIP Score': vip_scores
}).sort_values(by='VIP Score', ascending=False)

vip_df.reset_index(drop=True, inplace=True)

# 중요한 피처 선택 (VIP Score >= 0.9)
important_features = vip_df[vip_df['VIP Score'] >= 0.9]['Feature'].tolist()

# 출력: 선택된 피처들
print(f"선택된 중요 피처들 (VIP ≥ 0.9):")
print(vip_df[vip_df['VIP Score'] >= 0.9])

# 각 그룹별로 train/test split
X_train_all, X_test_all, y_train_all, y_test_all = [], [], [], []